<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%201/1_8_3_Kalifornien_Hauspreise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os # u.a. zur Entwicklugn plattformübergreifender Systempfade
import pandas as pd # Datenmanagement
import numpy as np # Hilfsfunktionen für mathematische Operationen

# Datenvisualisierung
import seaborn as sns 
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split # Datensplits
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn import metrics # Modellevaluierung

## eigene Funktionen
def filter_df_by_proximity(df, proximity):
    return df.loc[df["ocean_proximity"] == proximity]

def engineer_features(df):
    df["ratio_bedrooms"] = df["total_bedrooms"] / df["total_rooms"]
    df["people_per_household"] = df["population"] / df["households"]
    return df

In [ ]:
import os
import tarfile
import urllib.request
import shutil
import requests

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.getcwd()
CALIFORNIA_URL = "https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/california.png"
CALIFORNIA_PATH = "california.png"
FILE_PATH = "housing.csv"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

response = requests.get(CALIFORNIA_URL, stream=True)
with open(CALIFORNIA_PATH, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del response

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

df = pd.read_csv(FILE_PATH) # Wir lesen die Datei housing.csv ein

df = df.dropna() # löscht alle Zeile mit fehlenden Attributen
df = df.reset_index(drop=True) # zählt unsere Daten neu durch

description = df.describe()

bins = [0] + list(description["median_house_value"][
    ["25%", "50%", "75%"]
].astype(int)) + [np.inf]

df["house_cat"] = pd.cut(
    df["median_house_value"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in split.split(df, df["house_cat"]):
    df_train = df.loc[train_index]
    df_test = df.loc[test_index]
    
df_train = df_train.drop("house_cat", axis=1)
df_test = df_test.drop("house_cat", axis=1)

df_train = df_train.drop(filter_df_by_proximity(df_train, "ISLAND").index)
df_test = df_test.drop(filter_df_by_proximity(df_test, "ISLAND").index)

df_train = engineer_features(df_train)
df_test = engineer_features(df_test)

# 1.9 Hauspreise in Kalifornien

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_housing.jpg">

### Rohdaten in KI-Format bringen

Wir wollen nun unser erstes Machine Learning-Modell im Supervised Learning-Stil implementieren und trainieren. Schauen wir nochmal auf unsere Daten:

In [ ]:
df_train.head()

Wir haben mit `ocean_proximity` noch ein kategoriales Attribut. Die meisten ML-Algorithmen können jedoch nur numerischen Input verarbeiten; wir müssen diese also noch transformieren. Die einfachste Wahl ist hier eine One-Hot Transformation. Auch hier hilft uns `pandas`:

In [ ]:
df_train_ml = pd.get_dummies(df_train) # One-Hot Encoding
df_test_ml = pd.get_dummies(df_test)

Schauen wir uns die Daten erneut an:

In [ ]:
df_train_ml.head()

Wir haben nun nur noch numerische Daten in unseren Attributen. Es kann losgehen! :)
Wir werden unsere Daten nun in die Eingabenmatrix `X` und den Vektor `y` transformieren, aus denen unser Modell lernen wird. Wir bauen uns dafür eine Hilfsfunktion:

In [ ]:
def get_features_and_targets(df):
    X = df.drop(["median_house_value"], axis=1).values
    y = np.stack(df["median_house_value"])
    return X, y

Anschließend wenden wir unsere Funktion auf unsere Daten an:

In [ ]:
X_train, y_train = get_features_and_targets(df_train_ml)
X_test, y_test = get_features_and_targets(df_test_ml)

### KI-Implementierung und Training

Nun kommt der Part, auf den wir bisher gewartet haben; wir bauen und trainieren unser erstes KI-Modell:

In [ ]:
clf = LinearRegression()
clf.fit(X_train, y_train)

... das war tatsächlich schon die Implementierung + Training!


Zugegeben, eine lineare Regression gilt noch nicht als hochmoderner KI-Algorithmus - und selbst beim Training dieser Regression können wir noch einige Optimierungen durch Trainingskonfigurationen vornehmen. 

Nichtsdestotrotz ist dies unser erstes Modell, welches rein auf Basis von Daten in der Lage ist, zu lernen, um Progonsen zu erstellen. Mit `scikit-learn` gibt es eine hervorragende Python-Bibliothek, die eine Vielzahl von KI-Modellen bereitstellt.